#Trabajo práctico n°1#

###Moreira Pablo###




# Cargo librerías

In [ ]:
import numpy as np #manejo de arreglos

import matplotlib.pyplot as plt #gráficos

from sklearn.decomposition import PCA #Componentes principales
from sklearn.preprocessing import StandardScaler #Escalado de datos

from sklearn.naive_bayes import GaussianNB #clasificador bayesiano ingenuo
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  #discriminante lineal
from sklearn.linear_model import LogisticRegression #regresión logística

# matriz de confusión
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# métricas para evaluar la clasificación predicha
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report, plot_roc_curve
# divide en conjuntos para entrenar y para testeat

from sklearn.model_selection import train_test_split


# Tamaño de gráficos
plt.rcParams["figure.figsize"] = (10,10)

# Datos y exploración



In [ ]:
# cargamos datos desde el drive
from google.colab import drive
drive.mount('/content/drive')


# carpeta del drive
path = '/content/drive/MyDrive/UNAB Ciencia de Datos en uso/Inferencia estadística/TP1/'

filename = 'Breast Cancer Prediction1.csv'

cols_a_usar = (1,2,3,4,5,6,7,8,9,10) #saco la columna de codigo aleatorio que no sirve

datos = np.genfromtxt(path+filename, delimiter=',',usecols = cols_a_usar, skip_header=1) 


#como hay datos nulos, eliminamos todas estas filas con "nan"
datos = datos[~np.isnan(datos).any(axis=1)]

t = np.genfromtxt(path+filename, delimiter=',', usecols = cols_a_usar, max_rows=1, dtype=str) #genero los titulos
print (t)

X = datos[:,0:-1]
print(X.shape)
print(X[0:5,:])

y = datos[:,-1]
print(y.shape)
print(y[0:5])

# Separo en train/test. 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=88888)

# Standarizo datos de ENTRENAMIENTO
scaler = StandardScaler()
scaler.fit(X_train) #calcula promedio y desvío
X_train = scaler.transform(X_train) #estandariza con promedio y desvío anteriores

# Standarizo datos de TESTEO con los mismos promedio y desvío calculados para entrenamiento
X_test = scaler.transform(X_test)

# Standarizo datos en general con los mismos promedio y desvío calculados para entrenamiento
X= scaler.transform(X)

print(X_train.shape)
print(X_test.shape)


In [ ]:
# Escalo los datos
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

#Aplico Componentes principales
pca = PCA()
pca.fit(X,y)
x_new = pca.transform(X)
pca.components_[:,0] #vemos la influencia de las variables originales con las combinaciones lineales en pca.components_

In [ ]:
print([round(x,3) for x in pca.components_[:,0]])

In [ ]:
pca.components_[:,1] 

In [ ]:
print([round(x,3) for x in pca.components_[:,1]])

In [ ]:
#Varianza explicada por cada componente
print(pca.explained_variance_ratio_)
print([round(x,14) for x in pca.explained_variance_ratio_])

# Grafico las primeras dos componentes principales

In [ ]:
def myplot(score,coeff,labels=None,target_names=None,titulo1='Vista PCA',titulo2='Vista proyeccion var originales'):
     x1 = score[:,0]
     x2 = score[:,1]
     n = coeff.shape[0]

     clases = target_names
     scatter = plt.scatter(x1, x2, c=y, alpha=1) #, cmap=colours)
     plt.legend(handles=scatter.legend_elements()[0], labels=clases, fontsize=19)
     print(scatter.legend_elements())
     plt.legend(handles=scatter.legend_elements()[0], labels=scatter.legend_elements()[1], fontsize=16)

         
#Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");


#Grafico las primeras dos componentes principales 
myplot(x_new[:,0:2], pca.components_, t)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.grid()
plt.show()

In [ ]:
# Definimos el rango de los ejes del grafico
plt.axis([-1, 0.25, -0.5, 1.5])

# Vemos cuantos vectores son las direcciones de maxima varianza
n = pca.components_.shape[0]
print(n)
print(pca.components_)

# Recorremos esos vectores y los vamos dibujando en el plano
for i in range(n):
    plt.arrow(0, 0, pca.components_[i,0], pca.components_[i,1], color = 'g', alpha = 1)
    # En el extremo de cada vector ponemos en nombre de la columan correspondiente (un poco dezplazados)
    plt.text(pca.components_[i,0]*1.1 , pca.components_[i,1]*1.1, t[i], color = 'r', ha = 'center', va = 'center', fontsize=8)
    plt.xlim(None,0.5)
    plt.ylim(None, 0.5)

# Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");

# imprimimos el grafico completo
plt.show();

# Clasificador bayesiano ingenuo


In [ ]:
gnb = GaussianNB() #instancio la clase
modelo_gnb = gnb.fit(X_train, y_train) #entreno modelo predictivo a partir de los datos, es decir construyo en borde de decisión
y_pred_test = modelo_gnb.predict(X_test) #clasifico según modelo. Por defecto clasifica según la clase con probablidad más alta

# matriz de confusión
conf_g = confusion_matrix(y_test,y_pred_test)

# grafico matriz de confusión
disp_g = ConfusionMatrixDisplay(confusion_matrix=conf_g, display_labels=gnb.classes_)
disp_g.plot(values_format='d') 

# métricas de evaluación sobre los datos de TESTEO.
accuracy_g = accuracy_score(y_test, y_pred_test)
recall_g = recall_score(y_test, y_pred_test)
precision_g = precision_score(y_test, y_pred_test)

print('Métricas sobre datos de TEST')
print('Accuracy: ', round(accuracy_g,2))
print('Recall: ', round(recall_g,2))
print('Precision: ', round(precision_g,2))

In [ ]:
#implemento un punto de corte en 0.7
probas = modelo_gnb.predict_proba(X_test)

# por ejemplo, por defecto asumo que si termina en venta (y=1)
y_pred_test_custom = np.ones(y_test.shape)

#y si la probabilidad de y=0 es mayor a 0.7, lo clasifico como no venta (y=0)
for i in range(probas.shape[0]):
    if (probas[i,0]>0.2):
        y_pred_test_custom[i]=0.

# matriz de confusión
conf_custom = confusion_matrix(y_test,y_pred_test_custom)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf_custom, display_labels=gnb.classes_)
disp.plot(values_format='d') 

#métricas de evaluación sobre los datos de TESTEO, punto de corte arbitrario
accuracy_custom = accuracy_score(y_test, y_pred_test_custom)
recall_custom = recall_score(y_test, y_pred_test_custom)
precision_custom = precision_score(y_test, y_pred_test_custom)
f1_custom = f1_score(y_test, y_pred_test_custom)
clasif_report_custom = classification_report(y_test, y_pred_test_custom)

print('')
print('Métricas sobre datos nuevos de TEST, punto de corte arbitrario 0.7')
print('Accuracy: ', round(accuracy_custom,2))
print('Recall: ', round(recall_custom,2))
print('Precision: ', round(precision_custom,2))
print('F1: ', round(f1_custom,2))
print('Classification Report:')
print(clasif_report_custom)

Clasificador Análisis Discriminante Lineal

In [ ]:
lda = LinearDiscriminantAnalysis() #instancio la clase
modelo_lda = lda.fit(X_train, y_train) #entreno modelo predictivo a partir de los datos, es decir construyo en borde de decisión
y_pred_test_dl = modelo_lda.predict(X_test) #clasifico según modelo. Por defecto clasifica según la clase con probablidad más alta


# matriz de confusión
conf = confusion_matrix(y_test,y_pred_test_dl)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels=lda.classes_)
disp.plot(values_format='d') 

# métricas de evaluación sobre los datos de TESTEO.
accuracy_dl = accuracy_score(y_test, y_pred_test_dl)
recall_dl = recall_score(y_test, y_pred_test_dl)
precision_dl = precision_score(y_test, y_pred_test_dl)
f1_dl = f1_score(y_test, y_pred_test_dl)
clasif_report_dl = classification_report(y_test, y_pred_test_dl)


print('Métricas sobre datos de TEST sin punto de corte arbitrario')
print('Accuracy: ', round(accuracy_dl,2))
print('Recall: ', round(recall_dl,2))
print('Precision: ', round(precision_dl,2))
print('F1: ', round(f1_dl,2))
print('Classification Report:')
print(clasif_report_dl)

# Clasificador Regresión Logística



In [ ]:
lg = LogisticRegression() #instancio la clase
modelo_lg = lg.fit(X_train, y_train) 
# clasifico según el modelo, es decir predigo las clases, con los datos de TESTEO.
y_pred_test = modelo_lg.predict(X_test)

# matriz de confusión
conf = confusion_matrix(y_test,y_pred_test)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels=lg.classes_)
disp.plot(values_format='d') 

# métricas de evaluación sobre los datos de TESTEO.
accuracy = accuracy_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
clasif_report = classification_report(y_test, y_pred_test)


print('Métricas sobre datos de TEST sin punto de corte arbitrario')
print('Accuracy: ', round(accuracy,2))
print('Recall: ', round(recall,2))
print('Precision: ', round(precision,2))
print('F1: ', round(f1,2))
print('Classification Report:')
print(clasif_report)


# Coeficientes de la Regresión Logística

In [ ]:
lg.coef_

# Evaluación en TEST por Curva ROC

In [ ]:
plot_roc_curve(modelo_lg, X_test, y_test)

###Variando punto de corte


In [ ]:

#implemento un punto de corte en 0.7
probas = modelo_lg.predict_proba(X_test)

# por ejemplo, por defecto asumo que si termina en venta (y=1)
y_pred_test_custom = np.ones(y_test.shape)

#y si la probabilidad de y=0 es mayor a 0.7, lo clasifico como no venta (y=0)
for i in range(probas.shape[0]):
    if (probas[i,0]>0.2):
        y_pred_test_custom[i]=0.

# matriz de confusión
conf_custom = confusion_matrix(y_test,y_pred_test_custom)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf_custom, display_labels=lg.classes_)
disp.plot(values_format='d') 

#métricas de evaluación sobre los datos de TESTEO, punto de corte arbitrario
accuracy_custom = accuracy_score(y_test, y_pred_test_custom)
recall_custom = recall_score(y_test, y_pred_test_custom)
precision_custom = precision_score(y_test, y_pred_test_custom)
f1_custom = f1_score(y_test, y_pred_test_custom)
clasif_report_custom = classification_report(y_test, y_pred_test_custom)

print('')
print('Métricas sobre datos nuevos de TEST, punto de corte arbitrario 0.7')
print('Accuracy: ', round(accuracy_custom,2))
print('Recall: ', round(recall_custom,2))
print('Precision: ', round(precision_custom,2))
print('F1: ', round(f1_custom,2))
print('Classification Report:')
print(clasif_report_custom)

In [ ]:
plot_roc_curve(modelo_lg, X_test, y_pred_test_custom)

In [ ]:
#implemento un punto de corte en 0.7
probas_07 = modelo_lg.predict_proba(X_test)

# por ejemplo, por defecto asumo que si termina en venta (y=1)
y_pred_test_custom_07 = np.ones(y_test.shape)

#y si la probabilidad de y=0 es mayor a 0.7, lo clasifico como no venta (y=0)
for i in range(probas.shape[0]):
    if (probas[i,0]>0.7):
        y_pred_test_custom_07[i]=0.

# matriz de confusión
conf_custom_07 = confusion_matrix(y_test,y_pred_test_custom_07)

# grafico matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=conf_custom_07, display_labels=lg.classes_)
disp.plot(values_format='d') 

#métricas de evaluación sobre los datos de TESTEO, punto de corte arbitrario
accuracy_custom_07 = accuracy_score(y_test, y_pred_test_custom_07)
recall_custom_07 = recall_score(y_test, y_pred_test_custom_07)
precision_custom_07 = precision_score(y_test, y_pred_test_custom_07)
f1_custom_07 = f1_score(y_test, y_pred_test_custom_07)
clasif_report_custom_07 = classification_report(y_test, y_pred_test_custom_07)

print('')
print('Métricas sobre datos nuevos de TEST, punto de corte arbitrario 0.7')
print('Accuracy: ', round(accuracy_custom_07,2))
print('Recall: ', round(recall_custom_07,2))
print('Precision: ', round(precision_custom_07,2))
print('F1: ', round(f1_custom_07,2))
print('Classification Report:')
print(clasif_report_custom_07)

In [ ]:
plot_roc_curve(modelo_lg, X_test, y_pred_test_custom_07)

# Clasificador Regresión Logística sin usar train/test

In [ ]:
lg_base = LogisticRegression() #instancio la clase
modelo_lg_base =lg_base.fit(X, y) 
# clasifico según el modelo, es decir predigo las clases, con los datos de TESTEO.
y_pred_test_base = modelo_lg_base.predict(X)

# matriz de confusión
conf_base = confusion_matrix(y,y_pred_test_base)

# grafico matriz de confusión
disp_base = ConfusionMatrixDisplay(confusion_matrix=conf_base, display_labels=lg_base.classes_)
disp_base.plot(values_format='d') 

# métricas de evaluación sobre los datos de TESTEO.
accuracy = accuracy_score(y, y_pred_test_base)
recall = recall_score(y, y_pred_test_base)
precision = precision_score(y, y_pred_test_base)

print('Métricas sobre datos sin TEST')
print('Accuracy: ', round(accuracy,2))
print('Recall: ', round(recall,2))
print('Precision: ', round(precision,2))

# Coeficientes de la Regresión Logística sin usar train/test

In [ ]:
lg_base.coef_

Aplico componentes principales al modelo de regrecion logistica elegido como mejor 20% de datos para test y punto de corte 0.8

In [ ]:
pca_reglog = PCA()
pca_reglog.fit(X,y_pred_test_custom)
x_new_reglog = pca_reglog.transform(X)
pca_reglog.components_[:,0]

In [ ]:
print([round(x,3) for x in pca_reglog.components_[:,0]])

In [ ]:
pca_reglog.components_[:,1] 

In [ ]:
print([round(x,3) for x in pca_reglog.components_[:,1]])

In [ ]:
#Varianza explicada por cada componente
print(pca_reglog.explained_variance_ratio_)
print([round(x,3) for x in pca_reglog.explained_variance_ratio_])

# Grafico las primeras dos componentes principales

In [ ]:
def myplot(score,coeff,labels=None,target_names=None,titulo1='Vista PCA',titulo2='Vista proyeccion var originales'):
     x1 = score[:,0]
     x2 = score[:,1]
     n = coeff.shape[0]

     clases = target_names
     scatter = plt.scatter(x1, x2, c=y, alpha=1) #, cmap=colours)
     plt.legend(handles=scatter.legend_elements()[0], labels=clases, fontsize=19)
     print(scatter.legend_elements())
     plt.legend(handles=scatter.legend_elements()[0], labels=scatter.legend_elements()[1], fontsize=16)

         
#Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");


#Grafico las primeras dos componentes principales 
myplot(x_new_reglog[:,0:2], pca_reglog.components_, t)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.grid()
plt.show()

In [ ]:
# Definimos el rango de los ejes del grafico
plt.axis([-1,0.25,-0.5,1])

# Vemos cuantos vectores son las direcciones de maxima varianza
n_reglog = pca_reglog.components_.shape[0]
print(n_reglog)
print(pca_reglog.components_)

# Recorremos esos vectores y los vamos dibujando en el plano
for i in range(n):
    plt.arrow(0, 0, pca_reglog.components_[i,0], pca_reglog.components_[i,1], color = 'g', alpha = 1)
    # En el extremo de cada vector ponemos en nombre de la columan correspondiente (un poco dezplazados)
    plt.text(pca_reglog.components_[i,0]*1.1 , pca_reglog.components_[i,1]*1.1, t[i], color = 'r', ha = 'center', va = 'center', fontsize=8)
    plt.xlim(None,0.5)
    plt.ylim(None, 0.5)

# Le ponemos nombre a los ejes
plt.xlabel("PC1");
plt.ylabel("PC2");

# imprimimos el grafico completo
plt.show();